<a href="https://colab.research.google.com/github/bhadreshpsavani/ColabNotebooks/blob/master/Copy_of_FederatedLearningUsingMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
!pip install syft
import matplotlib.pyplot as plt
import numpy as np
import torch

from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

from sklearn.model_selection import train_test_split

     |████████████████████████████████| 215kB 2.8MB/s 
     |████████████████████████████████| 1.4MB 65.0MB/s 
     |████████████████████████████████| 204kB 66.2MB/s 
     |████████████████████████████████| 71kB 33.3MB/s 
     |████████████████████████████████| 450kB 68.5MB/s 
     |████████████████████████████████| 389kB 52.5MB/s 
     |████████████████████████████████| 256kB 70.5MB/s 
     |████████████████████████████████| 276kB 71.6MB/s 
     |████████████████████████████████| 51kB 21.3MB/s 
     |████████████████████████████████| 122kB 65.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
     

In [0]:
import torchvision.datasets as datasets
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)

  0%|          | 16384/9912422 [00:00<01:29, 111135.24it/s]

9920512it [00:00, 28946825.43it/s]                           


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 457624.93it/s]
  1%|          | 16384/1648877 [00:00<00:11, 145507.03it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 7130473.22it/s]                           
8192it [00:00, 176080.08it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
train_data = mnist_trainset.train_data
train_targets = mnist_trainset.train_labels
len(train_data)

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:43: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


60000

In [0]:
test_data=mnist_trainset.test_data
test_targets=mnist_trainset.test_labels
len(test_targets)

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


60000

In [0]:
import torch as th
import syft as sy 
hook=sy.TorchHook(th)
from torch import nn, optim

W0704 16:33:55.646041 140675861645184 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0704 16:33:55.658718 140675861645184 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
nitin=sy.VirtualWorker(hook,id="nitin")
vipin=sy.VirtualWorker(hook,id="vipin")
secure_worker=sy.VirtualWorker(hook, id="secure_worker")

In [0]:
nitin_traindata=train_data[0:len(train_data)//2].send(nitin)
nitin_trainlabels=train_targets[0:len(train_data)//2].send(nitin)
vipin_traindata=train_data[len(train_data)//2:].send(vipin)
vipin_trainlabels=train_targets[len(train_data)//2:].send(vipin)
print(len(nitin_traindata), len(nitin_trainlabels), len(vipin_traindata) , len(vipin_trainlabels))

30000 30000 30000 30000


In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5)
        self.conv3 = nn.Conv2d(32,64, kernel_size=5)
        self.fc1 = nn.Linear(3*3*64, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(F.max_pool2d(self.conv3(x),2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = x.view(-1,3*3*64 )
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [0]:
def fit(model, train_data, train_target):
    
    
    vipin_model=model.copy().send(vipin)
    vipin_optimizer = torch.optim.Adam(model.parameters(),lr=0.001, betas=(0.9,0.999)).send(vipin)
    vipin_error = nn.CrossEntropyLoss().send(vipin)
    
    EPOCHS = 5
    
    nitin_model.to(device);
    nitin_model.train()
    for epoch in range(EPOCHS):
        correct = 0
        for i in range(len(train_data)):
            X_batch, y_batch = train_data[i].to(device), train_target[i].to(device)
            var_X_batch = X_batch.float()
            var_y_batch = y_batch
            optimizer.zero_grad()
            output = nitin_model(var_X_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()

            # Total correct predictions
            predicted = torch.max(output.data, 1)[1] 
            correct += (predicted == var_y_batch).sum()
    
            #print(correct)
       
        print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
            epoch, batch_idx*len(X_batch), len(train_loader.dataset), 100.*batch_idx / len(train_loader), loss, float(correct*100) / float(BATCH_SIZE*(batch_idx+1))))
    return nitin_model, 

In [0]:
model=CNN()

#train_nitin
nitin_model=model.copy().send(nitin)
nitin_optimizer = torch.optim.Adam(model.parameters(),lr=0.001, betas=(0.9,0.999))
nitin_error = nn.CrossEntropyLoss()

EPOCHS = 5
    
nitin_model.to(device);
nitin_model.train()
for epoch in range(EPOCHS):
    correct = 0
    for i in range(len(train_data)):
        X_batch, y_batch = train_data[i].to(device), train_target[i].to(device)
        var_X_batch = X_batch.float()
        var_y_batch = y_batch
        nitin_optimizer.zero_grad()
        output = nitin_model(var_X_batch)
        loss = nitin_error(output, var_y_batch)
        loss.backward()
        nitin_optimizer.step()
        # Total correct predictions
        predicted = torch.max(output.data, 1)[1] 
        correct += (predicted == var_y_batch).sum()
        
    print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(epoch, i*len(X_batch), len(train_data), 100.*i / len(train_data), loss, float(correct*100)))

AttributeError: ignored